# introduction
This document mainly implements the replacement of selected keywords with synonyms and implements a Train-test split. It utilizes TF-IDF features and emotion scores to train a Logistic Regression model as the baseline model.

# 1. Import

In [152]:
import numpy as np
import pandas as pd
import re
import string
import seaborn as sns
from wordcloud import STOPWORDS
from PIL import Image
import matplotlib.pyplot as plt
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set =  set(stopwords.words('english'))
#we clean up unnecessary marks
from sklearn.metrics import classification_report
nltk.download('punkt')  # 第一次使用时需要下载
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laneylee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/laneylee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [167]:
result = pd.read_csv('result/all&suicide_with_prob/all_probs.csv')
result

,id,predictions,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,...,remorse,sadness,positive,neutral,class,text,token,sentiment,token_count,text_length
0,0,12.0,0.003140,0.013643,0.320729,0.105968,1.306766e-03,3.976417e-03,1.945175e-03,6.303866e-04,...,6.034370e-04,1.836595e-02,5.781246e-02,0.444644,almosthomeless,I literally don’t know where to ask this but I...,liter dont know ask serious question advic pls...,-0.063889,38.0,85.0
1,1,1.0,0.033857,0.866869,0.000738,0.083598,1.513196e-03,1.323251e-02,2.314404e-05,2.276647e-06,...,2.588070e-05,1.288332e-04,1.572072e-07,0.000009,almosthomeless,I'm almost 16 and am going to study med but I ...,im almost go studi med need take coach henc m...,0.136364,29.0,58.0
2,2,3.0,0.000549,0.299958,0.000882,0.639542,1.295966e-03,4.677313e-03,9.640865e-04,4.965249e-06,...,4.310008e-08,1.453962e-06,2.927398e-05,0.052072,almosthomeless,"I got a job, I go to orientation Tuesday. Tomo...",got job go orient tuesday tomorrow homeless we...,0.050451,83.0,171.0
3,3,1.0,0.002605,0.637379,0.008194,0.351810,3.153184e-08,6.816774e-07,1.871734e-08,3.432036e-09,...,1.738535e-09,2.225840e-08,6.092990e-07,0.000011,almosthomeless,Struggling over here to get everything lined u...,struggl get everyth line upim need refug seek ...,-0.002083,119.0,228.0
4,4,12.0,0.000372,0.001420,0.004618,0.008440,3.367238e-04,1.772046e-03,4.271182e-03,6.603977e-04,...,5.265729e-04,6.840330e-04,3.335223e-02,0.912119,almosthomeless,Ill be homeless officially Monday. Moving out ...,ill homeless offici monday move sunday hope st...,-0.100000,39.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14790,14047,3.0,0.000194,0.003100,0.272729,0.447826,5.954654e-03,1.449352e-03,4.251438e-02,3.976550e-03,...,4.438736e-03,8.515444e-02,3.094133e-03,0.071845,normal_positive,Luigi is the best character in super Mario 3D ...,luigi best charact super mario world disagre ...,0.666667,9.0,19.0
14791,14048,3.0,0.006142,0.040784,0.169939,0.467442,3.218773e-03,1.576891e-03,1.857177e-03,1.132213e-03,...,2.526511e-03,9.345067e-03,3.685362e-02,0.252023,normal_positive,"Man, rye bread with ham and cheese is really g...",man rye bread ham chees realli good tri sell r...,0.700000,12.0,22.0
14792,14049,1.0,0.234363,0.378303,0.111972,0.129934,6.946132e-02,4.589238e-02,1.125795e-04,7.796997e-04,...,4.606484e-03,2.616194e-03,1.306576e-02,0.008035,normal_positive,"Idk what to call this, but here’s a rant I thi...",idk call here rant think life go downhil never...,0.120000,30.0,53.0
14793,14050,1.0,0.003498,0.715970,0.043989,0.062357,6.876551e-02,7.745647e-02,1.038076e-04,1.041726e-04,...,3.678730e-03,2.400653e-02,1.788583e-05,0.000030,normal_positive,Important announcement! Axolotls are the best ...,import announc axolotl best creatur planetthat...,0.550000,9.0,18.0


In [169]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[^a-zA-Z\s]', '', text)# filter non English char
    text = [word for word in text.split(' ') if word not in stopwords_set]
    text=" ".join(text)
#     text = [stemmer.stem(word) for word in text.split(' ')]
#     text=" ".join(text)
    return text

In [170]:
result['cleaned_text'] = result['text'].apply(clean)

In [155]:
keywords = {
    'adhd': ['adhd', 'vyvanse','medication', 'diagnosed'], 
    'almosthomeless': [ 'homeless', 'help', 'eviction', 'car'],
    'anxiety': ['anxiety', 'medication'],
    'assistance': ['assist', 'help',  'money'],
    'bipolar': ['bipolar',  'depression'],
    'depression': ['depression'],
    'eatingdisorders': [ 'foods', 'weight', 'disorder', 'eating lot', 'ate lot'],
    'get_motivated': ['energy', 'outcome','motivate'],
#     'normal_positive': ['good', 'best', 'happy', 'birthday', 'friend'],
    'ocd': [ 'ocd',  'obsessive'],
    'ptsd': [ 'therapist'],
    'selfharm': ['cut', 'self harm', 'blade','harm'],
    'stress': ['stress'],
    #'mask':['doubt', 'depress', 'trouble', 'pressure', 'sadness', 'grief'], # --> from paper, may not suit our purpose
    'suicide':['die','death','survive','suicide','kill']
}

In [163]:
mental_class = ['adhd',  'anxiety', 'bipolar', 'depression', 'eatingdisorders',  'ocd', 'ptsd', 'normal_positive','selfharm', 'stress']

In [162]:
# result = result.loc[result['class'].isin(mental_class)]

In [164]:
result = result.reset_index(drop = True)

In [165]:
result

,id,predictions,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,...,neutral,class,text,token,sentiment,token_count,text_length,cleaned_text,text_no_kw,cleaned_text_no_kw
0,907,1.0,0.030109,0.935445,0.012993,0.020151,9.139813e-06,0.001271,1.248578e-05,2.741102e-06,...,0.000006,anxiety,throwaway because i don’t want my main account...,throwaway dont want main account connect birt...,-0.006410,99.0,204.0,throwaway dont want main account connected bi...,throwaway because i don’t want my main account...,throwaway dont want main account connected bi...
1,908,8.0,0.000124,0.000487,0.013218,0.000274,1.135681e-03,0.002319,4.072795e-01,1.077160e-04,...,0.000245,anxiety,Hey guys Im having a rough start in the mornin...,hey guy im rough start morn girlfriend leav go...,-0.100000,28.0,61.0,hey guys im rough start morning girlfriend lea...,hey guys im having a rough start in the mornin...,hey guys im rough start morning girlfriend lea...
2,909,1.0,0.021668,0.895212,0.005925,0.069638,1.147155e-03,0.005089,1.872910e-05,3.701496e-07,...,0.001293,anxiety,I run a small business since the beginning of ...,run small busi sinc begin year anyth turn shit...,-0.022917,74.0,155.0,run small business since beginning year anythi...,i run a small business since the beginning of ...,run small business since beginning year anythi...
3,910,2.0,0.000130,0.014770,0.865925,0.119000,2.723221e-12,0.000116,3.319793e-09,2.075076e-10,...,0.000058,anxiety,i went on a double date and i thought it went ...,went doubl date thought went well fun first st...,0.003571,129.0,280.0,went double date thought went well fun first s...,i went on a double date and i thought it went ...,went double date thought went well fun first s...
4,911,1.0,0.000040,0.857727,0.001016,0.100299,1.048685e-07,0.036433,1.759269e-03,1.191234e-06,...,0.002690,anxiety,Hey everyone. I’m 31 and have adhd along with ...,hey everyon im adhd along mood disord suffer ...,0.041667,88.0,192.0,hey everyone im adhd along mood disorders suf...,hey everyone. i’m 31 and have [mask] along wit...,hey everyone im [mask] along mood disorders s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11541,14047,3.0,0.000194,0.003100,0.272729,0.447826,5.954654e-03,0.001449,4.251438e-02,3.976550e-03,...,0.071845,normal_positive,Luigi is the best character in super Mario 3D ...,luigi best charact super mario world disagre ...,0.666667,9.0,19.0,luigi best character super mario world disagr...,luigi is the best character in super mario 3d ...,luigi best character super mario world disagr...
11542,14048,3.0,0.006142,0.040784,0.169939,0.467442,3.218773e-03,0.001577,1.857177e-03,1.132213e-03,...,0.252023,normal_positive,"Man, rye bread with ham and cheese is really g...",man rye bread ham chees realli good tri sell r...,0.700000,12.0,22.0,man rye bread ham cheese really good try sell ...,"man, rye bread with ham and cheese is really g...",man rye bread ham cheese really good try sell ...
11543,14049,1.0,0.234363,0.378303,0.111972,0.129934,6.946132e-02,0.045892,1.125795e-04,7.796997e-04,...,0.008035,normal_positive,"Idk what to call this, but here’s a rant I thi...",idk call here rant think life go downhil never...,0.120000,30.0,53.0,idk call heres rant think life going downhill ...,"idk what to call this, but here’s a rant i thi...",idk call heres rant think life going downhill ...
11544,14050,1.0,0.003498,0.715970,0.043989,0.062357,6.876551e-02,0.077456,1.038076e-04,1.041726e-04,...,0.000030,normal_positive,Important announcement! Axolotls are the best ...,import announc axolotl best creatur planetthat...,0.550000,9.0,18.0,important announcement axolotls best creature ...,important announcement! axolotls are the best ...,important announcement axolotls best creature ...


# 2. Train test split

In [171]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer

from sklearn.compose import ColumnTransformer

def train_multiclass_model_withemo(df_all,text = 'cleaned_text',if_numerical = True):
    df_all = df_all.dropna()
    # feature and label
    features = [ 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral',
         text]  
    label = 'class'


    X_train, X_test, y_train, y_test = train_test_split(
        df_all[features],  # feature
        df_all[label],  # label
        test_size=0.2,
        random_state=42
    )

    # define transformer for text and numerical features
    text_features = text 
    numerical_features = [ 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral']  #
    if if_numerical:
        preprocessor = ColumnTransformer(
            transformers=[
                ('text', TfidfVectorizer(), text_features),
                ('num', MinMaxScaler(), numerical_features) #('num', StandardScaler(),numerical_features)
            ]
        )
    else:
        preprocessor = ColumnTransformer(
        transformers=[
            ('text', TfidfVectorizer(), text_features)
            
        ]
    )
    # （'multinomial'）for multi-class and l2 regularization
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

    # Pipeline
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', clf)
    ])

    # Train
    model.fit(X_train, y_train)

    # Test
    y_pred = model.predict(X_test)
     # print accuray
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy：", accuracy)


    # average='macro'
    precision = precision_score(y_test, y_pred, average='macro')
    print("Precision", precision)

    recall = recall_score(y_test, y_pred, average='macro')
    print("Recall", recall)

    f1 = f1_score(y_test, y_pred, average='macro')
    print("F1-score：", f1)
    return X_train, X_test, y_train, y_test # need the train_test for validation!
    
X_train, X_test, y_train, y_test  = train_multiclass_model_withemo(result,if_numerical = False)# without emotion score

Accuracy： 0.7515781656145563
Precision 0.7607434975637773
Recall 0.7060655529746229
F1-score： 0.7260592104817327


In [172]:
train_multiclass_model_withemo(result,if_numerical = True) # with emotion score

Accuracy： 0.7486075009283327
Precision 0.7498437008723177
Recall 0.7044833088818947
F1-score： 0.7219183432890778


(          anger  annoyance  disappointment  disapproval       disgust  \
 12355  0.000377   0.016068        0.035070     0.202846  7.202113e-04   
 14364  0.008999   0.056374        0.166283     0.652791  4.573573e-04   
 1346   0.014021   0.190706        0.130012     0.106261  4.239085e-04   
 7541   0.007742   0.256413        0.003211     0.731839  1.681962e-07   
 8221   0.000031   0.026559        0.092793     0.238683  2.910692e-05   
 ...         ...        ...             ...          ...           ...   
 6051   0.000294   0.001740        0.257945     0.173793  1.649276e-04   
 14752  0.008455   0.093432        0.013903     0.061975  8.909390e-03   
 6309   0.248555   0.542076        0.009714     0.197001  1.931459e-05   
 1037   0.000345   0.998720        0.000035     0.000524  8.308086e-09   
 8440   0.050746   0.081933        0.030692     0.001583  1.811520e-01   
 
        embarrassment          fear         grief   nervousness       remorse  \
 12355       0.000912  4.7974

In [38]:
pd.concat([X_train,y_train],axis = 1).reset_index(drop = True).to_csv('result/df_all_train.csv') # save train set

## 3. Text Augmentation

In [392]:
# replace_kw = keywords_synonyms 

In [19]:
replace_kw = { # suggest by Chatgpt
  'adhd': {
    'adhd': ['Attention Deficit Hyperactivity Disorder'],
    'vyvanse': ['lisdexamfetamine', 'stimulant medication'],
    'medication': ['medicine', 'drug', 'treatment'],
    'diagnosed': ['identified', 'detected', 'recognized']
  },
  'almosthomeless': {
    'homeless': ['unhoused', 'homelessness', 'displaced'],
    'help': ['assist', 'aid', 'support'],
    'eviction': ['displacement', 'expulsion', 'removal'],
    'car': ['automobile', 'vehicle', 'motorcar']
  },
  'anxiety': {
    'anxiety': ['nervousness', 'worry', 'apprehension'],
    'medication': ['medicine', 'drug', 'treatment']
  },
  'assistance': {
    'assist': ['help', 'aid', 'support'],
    'help': ['assist', 'aid', 'support'],
    'money': ['cash', 'funds', 'capital']
  },
  'bipolar': {
    'bipolar': ['manic-depressive', 'bipolar disorder'],
    'depression': ['sadness', 'melancholy', 'despondency']
  },
  'depression': {
    'depression': ['sadness', 'melancholy', 'despondency']
  },
  'eatingdisorders': {
    'foods': ['meals', 'edibles', 'nourishment'],
    'weight': ['mass', 'heaviness', 'bulk'],
    'disorder': ['condition', 'disease', 'ailment'],
    'eating lot': ['overeating', 'binge eating'],
    'ate lot': ['overate', 'binged']
  },
  'get_motivated': {
    'energy': ['vitality', 'vigor', 'dynamism'],
    'outcome': ['result', 'consequence', 'effect'],
    'motivate': ['inspire', 'encourage', 'stimulate']
  },
  'ocd': {
    'ocd': ['Obsessive-Compulsive Disorder', 'compulsion'],
    'obsessive': ['compulsive', 'fixated', 'obsessed']
  },
  'ptsd': {
    'therapist': ['counselor', 'psychotherapist', 'psychologist']
  },
  'selfharm': {
    'cut': ['slice', 'slash', 'gash'],
    'self harm': ['self-injury', 'self-mutilation'],
    'blade': ['knife', 'razor', 'cutter'],
    'harm': ['injury', 'damage', 'hurt']
  },
  'stress': {
    'stress': ['pressure', 'tension', 'strain']
  },
  'suicide': {
    'die': ['pass away', 'perish', 'expire'],
    'death': ['demise', 'fatality', 'mortality'],
    'survive': ['live through', 'outlast', 'endure'],
    'suicide': ['self-destruction', 'self-killing'],
    'kill': ['murder', 'slay', 'assassinate'],
    'anxiety': ['nervousness', 'worry', 'apprehension'],
    'medication': ['medicine', 'drug', 'treatment'],
      'harm': ['injury', 'damage', 'hurt'],
      'help': ['assist', 'aid', 'support']
      
  }
}
# work with 1-gram!

In [180]:
import random

def replace_sys(word,text,synonyms,row,new_rows):
    try:
        if word in text.lower():
            # Randomly decide whether to replace and which synonym to use
            if random.random() < 0.7:  # 70% chance of replace
                synonym = random.choice(synonyms)  # randomly choose synonyms
                new_text = re.sub(r'\b{i}\b'.format(i=word), synonym, text, flags=re.IGNORECASE)
                # add new row
                new_row = row.copy()
                new_row['cleaned_text'] = new_text  
                new_rows.append(new_row)  
    except:
        pass

def augment_df(result,category):
    synonyms_dict = replace_kw[category]


    category_rows = result[result['class'] == category]


    new_rows = []

    for index, row in category_rows.iterrows():
        text = row['cleaned_text']  
        for word in synonyms_dict.keys():
            synonyms = synonyms_dict[word]
            print(word,synonyms)
            replace_sys(word,text,synonyms,row,new_rows)


    new_rows_df = pd.DataFrame(new_rows)
    new_rows_df['augmented'] = 1
    new_rows_df.reset_index(drop=True, inplace=True)

    return new_rows_df


In [181]:
Train = pd.concat([X_train,y_train],axis = 1)
Train = Train.reset_index(drop=True)
Train['augmented'] = 0 # 标记是否为增强
Train

,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,nervousness,remorse,sadness,positive,neutral,cleaned_text,class,augmented
0,0.000377,0.016068,0.035070,0.202846,7.202113e-04,0.000912,4.797461e-03,3.573275e-04,1.274492e-02,2.495390e-04,2.017067e-03,4.202968e-01,3.035441e-01,great news father recovering corona hospital ...,normal_positive,0
1,0.008999,0.056374,0.166283,0.652791,4.573573e-04,0.001685,1.878334e-05,2.809609e-04,4.235959e-04,6.475875e-04,8.465126e-03,9.562749e-02,7.947659e-03,good morning people put cereal first oh also g...,normal_positive,0
2,0.014021,0.190706,0.130012,0.106261,4.239085e-04,0.554364,4.157982e-04,1.068727e-05,2.324627e-03,2.911549e-07,3.717595e-04,4.290392e-06,1.084685e-03,first time sub ive lexapro days took panic an...,anxiety,0
3,0.007742,0.256413,0.003211,0.731839,1.681962e-07,0.000792,3.692591e-07,2.726226e-10,2.304939e-11,2.929336e-10,3.892165e-09,5.178214e-11,2.781450e-06,throwaway account im embarrassed talk thishi i...,adhd,0
4,0.000031,0.026559,0.092793,0.238683,2.910692e-05,0.165362,3.603083e-02,1.113617e-04,3.088459e-01,1.150014e-05,1.333104e-02,1.139902e-04,1.180985e-01,hey see lot posts afraid share diagnosis other...,bipolar,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10763,0.000294,0.001740,0.257945,0.173793,1.649276e-04,0.000888,3.172238e-04,7.567086e-04,5.395566e-01,8.791842e-05,2.104666e-02,1.363963e-04,3.273481e-03,lost mum october sunday gone got call say part...,stress,0
10764,0.008455,0.093432,0.013903,0.061975,8.909390e-03,0.004420,2.223132e-03,1.478796e-03,1.503773e-03,5.477535e-03,1.523704e-02,6.347883e-01,1.481973e-01,favourite forbidden snack kid mine wall also l...,normal_positive,0
10765,0.248555,0.542076,0.009714,0.197001,1.931459e-05,0.000025,1.810165e-04,6.183177e-06,8.342523e-07,1.268263e-08,1.591701e-04,3.819113e-07,2.262851e-03,something extremely stupidabout something alre...,stress,0
10766,0.000345,0.998720,0.000035,0.000524,8.308086e-09,0.000376,1.297842e-07,5.130466e-09,7.288504e-11,1.147119e-11,4.563700e-11,2.602309e-10,9.876947e-08,take wellbutrin social anxiety adhd gad thoug...,anxiety,0


In [182]:
df_augmented = Train.copy()

for i in mental_class:
    if i == 'normal_positive':
        continue
    new_rows_df = augment_df(df_augmented,i)
    df_augmented = pd.concat([df_augmented, new_rows_df], ignore_index=True)

adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd 

cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'damage', 'hurt']
cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'damage', 'hurt']
cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'damage', 'hurt']
cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'damage', 'hurt']
cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'damage', 'hurt']
cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'damage', 'hurt']
cut ['slice', 'slash', 'gash']
self harm ['self-injury', 'self-mutilation']
blade ['knife', 'razor', 'cutter']
harm ['injury', 'da

In [183]:
df_augmented = pd.concat([Train, df_augmented], ignore_index=True)

In [184]:
df_augmented.columns

Index(['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness',
       'positive', 'neutral', 'cleaned_text', 'class', 'augmented'],
      dtype='object')

In [185]:
df_augmented = df_augmented.drop_duplicates().dropna().reset_index(drop=True)

In [186]:
df_augmented

,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,nervousness,remorse,sadness,positive,neutral,cleaned_text,class,augmented
0,0.000377,0.016068,0.035070,0.202846,7.202113e-04,0.000912,4.797461e-03,3.573275e-04,1.274492e-02,2.495390e-04,2.017067e-03,4.202968e-01,3.035441e-01,great news father recovering corona hospital ...,normal_positive,0
1,0.008999,0.056374,0.166283,0.652791,4.573573e-04,0.001685,1.878334e-05,2.809609e-04,4.235959e-04,6.475875e-04,8.465126e-03,9.562749e-02,7.947659e-03,good morning people put cereal first oh also g...,normal_positive,0
2,0.014021,0.190706,0.130012,0.106261,4.239085e-04,0.554364,4.157982e-04,1.068727e-05,2.324627e-03,2.911549e-07,3.717595e-04,4.290392e-06,1.084685e-03,first time sub ive lexapro days took panic an...,anxiety,0
3,0.007742,0.256413,0.003211,0.731839,1.681962e-07,0.000792,3.692591e-07,2.726226e-10,2.304939e-11,2.929336e-10,3.892165e-09,5.178214e-11,2.781450e-06,throwaway account im embarrassed talk thishi i...,adhd,0
4,0.000031,0.026559,0.092793,0.238683,2.910692e-05,0.165362,3.603083e-02,1.113617e-04,3.088459e-01,1.150014e-05,1.333104e-02,1.139902e-04,1.180985e-01,hey see lot posts afraid share diagnosis other...,bipolar,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13788,0.000545,0.166072,0.000784,0.828500,1.060452e-04,0.000113,6.077322e-05,8.454267e-06,4.193470e-05,4.195828e-06,3.594980e-03,1.576899e-05,1.532859e-04,hi im years living lot tension always tension ...,stress,1
13789,0.209872,0.711447,0.007341,0.071199,1.948812e-05,0.000025,4.229647e-08,2.109629e-05,3.952906e-09,1.381709e-08,6.775095e-05,6.974235e-06,9.122931e-07,found dealing heightened tension anxiety becom...,stress,1
13790,0.000959,0.045486,0.254379,0.493968,3.246395e-04,0.001441,4.279843e-03,8.907627e-05,2.159943e-02,7.146539e-07,4.351602e-02,2.794524e-04,1.336785e-01,im financially stable dont really study hard d...,stress,1
13791,0.001555,0.009520,0.037964,0.021233,2.047394e-03,0.015122,4.014706e-05,1.170717e-05,2.921906e-03,1.050733e-06,1.778206e-04,7.150104e-02,8.379056e-01,hello long time see another rstress posti want...,stress,1


In [187]:
df_augmented.groupby('class').describe()['augmented']

,count,mean,std,min,25%,50%,75%,max
class,,,,,,,,
adhd,1355.0,0.427306,0.494870,0.0,0.0,0.0,1.0,1.0
almosthomeless,652.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
anxiety,1198.0,0.358932,0.479887,0.0,0.0,0.0,1.0,1.0
assistance,711.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
bipolar,1062.0,0.265537,0.441827,0.0,0.0,0.0,1.0,1.0
depression,917.0,0.152672,0.359867,0.0,0.0,0.0,0.0,1.0
eatingdisorders,1239.0,0.372881,0.483766,0.0,0.0,0.0,1.0,1.0
get_motivated,352.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
normal_positive,2161.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [188]:
y_test.value_counts()

class
normal_positive    516
ocd                209
adhd               207
depression         202
eatingdisorders    196
anxiety            195
ptsd               195
selfharm           194
bipolar            190
assistance         174
almosthomeless     173
stress             151
get_motivated       91
Name: count, dtype: int64

In [192]:
def LR_with_emo_feature(df_augmented,y_test,X_test,if_numerical = True):

    text_features = 'cleaned_text'  
    numerical_features = [ 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral'] 



    X_train, y_train = df_augmented[['anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral','cleaned_text']],df_augmented['class']
    if if_numerical:
        preprocessor = ColumnTransformer(
            transformers=[
                ('text', TfidfVectorizer(), text_features),
                ('num', MinMaxScaler(), numerical_features)
            ]
        )
    else:
        preprocessor = ColumnTransformer(
            transformers=[
                ('text', TfidfVectorizer(), text_features)
            ]
        )


    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')


    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', clf)
    ])


    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test) # return report, y-pred label and y-prob as probability

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy", accuracy)
    precision = precision_score(y_test, y_pred, average='macro')
    print("Precision", precision)

    recall = recall_score(y_test, y_pred, average='macro')
    print("Recall", recall)

    f1 = f1_score(y_test, y_pred, average='macro')
    print("F1-score：", f1)

    report = classification_report(y_test, y_pred, target_names=df_augmented['class'].unique())
#     print(report)
    return report,y_pred,y_prob

In [189]:
Train['class'].value_counts()

class
normal_positive    2183
ocd                 783
ptsd                783
bipolar             781
eatingdisorders     780
depression          779
adhd                776
anxiety             768
selfharm            768
assistance          716
almosthomeless      653
stress              644
get_motivated       354
Name: count, dtype: int64

In [71]:
# result_with_score,y_pred = LR_with_emo_feature(df_augmented,y_test,X_test,if_numerical = True) # quite random

In [193]:
result_with_score,y_pred,y_prob = LR_with_emo_feature(Train,y_test,X_test,if_numerical = False)

Accuracy 0.7515781656145563
Precision 0.7607434975637773
Recall 0.7060655529746229
F1-score： 0.7260592104817327


In [194]:
# def labelMapping(key):
#     label_dict = {
#     'normal_positive': 0,
#     'ocd': 1,
#     'adhd': 2,
#     'ptsd': 3,
#     'bipolar': 4,
#     'eatingdisorders': 5,
#     'depression': 6,
#     'selfharm': 7,
#     'anxiety': 8,
#     'stress': 9
# }
#     return label_dict[key]

# def reverseLabel(key):
#     label_dict = {
#     0:'normal_positive',
#     1:'ocd',
#     2:'adhd',
#     3:'ptsd',
#     4:'bipolar',
#     5:'eatingdisorders',
#     6:'depression',
#     7:'selfharm',
#     8:'anxiety',
#     9:'stress'
# }
#     return label_dict[key]

In [195]:
# y_pred_binary = to_categorical(pd.DataFrame(y_pred)[0].apply(labelMapping),num_classes = 10)
# y_true_binary = to_categorical(y_test.apply(labelMapping),num_classes = 10)

In [196]:
# from sklearn.metrics import roc_curve, auc,roc_auc_score
# from sklearn.metrics import roc_curve, auc
# from sklearn.preprocessing import label_binarize
# from tensorflow.keras.utils import to_categorical
# # after model training.....

# # average ROC&AUC
# roc_auc_macro = roc_auc_score(y_true_binary,y_prob , multi_class='ovr',average='macro')# one vs rest
# roc_auc_ovo = roc_auc_score(y_true_binary, y_prob, multi_class='ovr', average='weighted') 

# print('Macro-average ROC AUC:', roc_auc_macro)
# print('Weighted-average ROC AUC:', roc_auc_ovo)


# n_classes = 10

# # ROC and AUC for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(n_classes):
#     fpr[i], tpr[i], _ = roc_curve(y_true_binary[:, i], y_prob[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

# # Draw ROC Curve
# plt.figure()
# colors = ['aqua', 'darkorange', 'cornflowerblue', 'deeppink', 'limegreen',
#           'lightcoral', 'gold', 'blueviolet', 'olivedrab', 'peru'] # adjust with number of classes!

# for i, color in zip(range(n_classes), colors):
#     plt.plot(fpr[i], tpr[i], color=color, lw=2,
#              label='ROC curve of class {0} (area = {1:0.2f})'
#              ''.format(reverseLabel(i), roc_auc[i]))

# plt.plot([0, 1], [0, 1], 'k--', lw=2)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Multi-class ROC')
# plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
# plt.show()



In [34]:
df_augmented.to_csv('result/df_all_augmented.csv',index = False) # save augmented dataset

In [35]:
pd.concat([X_test,y_test],axis = 1).to_csv('result/df_all_test.csv',index = False) # save testing dataset

# 4. Do the same for suicide dataset


In [213]:
suicide = pd.read_csv('result/all&suicide_with_prob/suicide_probs.csv', engine = 'python',on_bad_lines='skip')

suicide

,id,predictions,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,...,remorse,sadness,positive,neutral,text,class,token,token_count,sentiment,text_count
0,0,12.0,0.003140,0.013643,0.320729,0.105968,1.306766e-03,3.976417e-03,1.945175e-03,6.303866e-04,...,6.034370e-04,1.836595e-02,5.781246e-02,0.444644,Hi guys I just wanted to say hi to you guys im...,non-suicide,hi guy want say hi guy im take note energi mat...,14.0,0.285714,23.0
1,1,1.0,0.033857,0.866869,0.000738,0.083598,1.513196e-03,1.323251e-02,2.314404e-05,2.276647e-06,...,2.588070e-05,1.288332e-04,1.572072e-07,0.000009,I'm gonna move and loose all my friends...agai...,suicide,im gonna move loos friendsagaini cant school l...,96.0,0.122727,212.0
2,2,3.0,0.000549,0.299958,0.000882,0.639542,1.295966e-03,4.677313e-03,9.640865e-04,4.965249e-06,...,4.310008e-08,1.453962e-06,2.927398e-05,0.052072,Heart set for New Year's.I won't fail this tim...,suicide,heart set new yearsi wont fail time ill free a...,23.0,-0.077273,39.0
3,3,1.0,0.002605,0.637379,0.008194,0.351810,3.153184e-08,6.816774e-07,1.871734e-08,3.432036e-09,...,1.738535e-09,2.225840e-08,6.092990e-07,0.000011,literally everyone sucks No one actually helps...,non-suicide,liter everyon suck one actual help matter say ...,13.0,0.000000,24.0
4,4,12.0,0.000372,0.001420,0.004618,0.008440,3.367238e-04,1.772046e-03,4.271182e-03,6.603977e-04,...,5.265729e-04,6.840330e-04,3.335223e-02,0.912119,"I'm UnlovableSeriously, I just know it. I've a...",suicide,im unlovableseri know ive alway felt like lose...,84.0,-0.024306,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13691,13629,12.0,0.000316,0.007487,0.216190,0.220375,7.305314e-05,4.107539e-02,5.660241e-03,2.327391e-04,...,1.653915e-02,1.121247e-02,3.433020e-02,0.255601,"I've seen what life has to offer, and it's not...",suicide,ive seen life offer mei thought go write big v...,11.0,0.000000,23.0
13692,13630,12.0,0.003728,0.015262,0.056664,0.018778,3.027335e-04,3.026527e-03,3.720161e-04,8.654860e-05,...,1.395357e-04,1.332107e-03,3.711219e-02,0.849442,Does anyone's anxiety make them suicidal?It's ...,suicide,anyon anxieti make suicidalit main reason give...,13.0,0.166667,28.0
13693,13631,12.0,0.001409,0.006077,0.064327,0.036853,7.010169e-03,1.005854e-02,3.020322e-02,6.443719e-03,...,5.932183e-03,1.802085e-02,1.186095e-01,0.441924,I have imagined my suicide for years. And I fe...,suicide,imagin suicid year feel final timefor entir li...,120.0,0.020539,263.0
13694,13632,1.0,0.053517,0.733234,0.026656,0.016210,2.811231e-02,1.234396e-02,9.731897e-04,1.210160e-04,...,1.099101e-03,7.398955e-03,1.135669e-01,0.006469,Why do people think its hard to be straight? A...,non-suicide,peopl think hard straight appar peopl think st...,13.0,0.036111,22.0


In [215]:
suicide['cleaned_text'] = suicide['text'].apply(clean)

In [216]:
X_train_sui, X_test_sui, y_train_sui, y_test_sui  = train_multiclass_model_withemo(suicide,if_numerical = False)# with emotional

Accuracy： 0.9294653014789533
Precision 0.9296104150352389
Recall 0.9293211883375818
F1-score： 0.9294224196580381


In [ ]:
df_suicide_train.to_csv('result/df_suicide_train.csv',index=False)

In [217]:
df_suicide_train = pd.concat([X_train_sui,y_train_sui],axis = 1).drop_duplicates().dropna()
df_suicide_ag = df_suicide_train.copy()

In [218]:
new_rows_df = augment_df(df_suicide_ag,'suicide')
df_suicide_ag= pd.concat([df_suicide_ag, new_rows_df], ignore_index=True).drop_duplicates().dropna()
df_suicide_ag = pd.concat([df_suicide_train, df_suicide_ag], ignore_index=True)

die ['pass away', 'perish', 'expire']
death ['demise', 'fatality', 'mortality']
survive ['live through', 'outlast', 'endure']
suicide ['self-destruction', 'self-killing']
kill ['murder', 'slay', 'assassinate']
anxiety ['nervousness', 'worry', 'apprehension']
medication ['medicine', 'drug', 'treatment']
harm ['injury', 'damage', 'hurt']
help ['assist', 'aid', 'support']
die ['pass away', 'perish', 'expire']
death ['demise', 'fatality', 'mortality']
survive ['live through', 'outlast', 'endure']
suicide ['self-destruction', 'self-killing']
kill ['murder', 'slay', 'assassinate']
anxiety ['nervousness', 'worry', 'apprehension']
medication ['medicine', 'drug', 'treatment']
harm ['injury', 'damage', 'hurt']
help ['assist', 'aid', 'support']
die ['pass away', 'perish', 'expire']
death ['demise', 'fatality', 'mortality']
survive ['live through', 'outlast', 'endure']
suicide ['self-destruction', 'self-killing']
kill ['murder', 'slay', 'assassinate']
anxiety ['nervousness', 'worry', 'apprehension

die ['pass away', 'perish', 'expire']
death ['demise', 'fatality', 'mortality']
survive ['live through', 'outlast', 'endure']
suicide ['self-destruction', 'self-killing']
kill ['murder', 'slay', 'assassinate']
anxiety ['nervousness', 'worry', 'apprehension']
medication ['medicine', 'drug', 'treatment']
harm ['injury', 'damage', 'hurt']
help ['assist', 'aid', 'support']
die ['pass away', 'perish', 'expire']
death ['demise', 'fatality', 'mortality']
survive ['live through', 'outlast', 'endure']
suicide ['self-destruction', 'self-killing']
kill ['murder', 'slay', 'assassinate']
anxiety ['nervousness', 'worry', 'apprehension']
medication ['medicine', 'drug', 'treatment']
harm ['injury', 'damage', 'hurt']
help ['assist', 'aid', 'support']
die ['pass away', 'perish', 'expire']
death ['demise', 'fatality', 'mortality']
survive ['live through', 'outlast', 'endure']
suicide ['self-destruction', 'self-killing']
kill ['murder', 'slay', 'assassinate']
anxiety ['nervousness', 'worry', 'apprehension

In [219]:
df_suicide_ag.drop_duplicates().dropna()

,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,nervousness,remorse,sadness,positive,neutral,cleaned_text,class,augmented
10547,0.029720,0.195006,0.076497,0.090729,2.581012e-02,7.298528e-03,8.623154e-02,1.277198e-03,8.681720e-02,4.382442e-04,0.026807,8.774608e-02,0.285623,interested inputi guess would anything part le...,suicide,1.0
10548,0.000668,0.002923,0.003512,0.001614,9.160416e-05,6.135543e-04,1.304268e-04,2.877059e-05,6.860851e-04,2.132672e-04,0.000293,9.631425e-01,0.026084,life opposite everything wish wasi admit privi...,suicide,1.0
10549,0.000433,0.084390,0.007133,0.903439,6.390168e-05,1.467594e-04,6.271207e-04,2.815610e-06,6.533143e-06,1.321671e-05,0.003479,1.611944e-07,0.000266,born female trans friend recently attempted s...,suicide,1.0
10550,0.000433,0.084390,0.007133,0.903439,6.390168e-05,1.467594e-04,6.271207e-04,2.815610e-06,6.533143e-06,1.321671e-05,0.003479,1.611944e-07,0.000266,born female trans friend recently attempted s...,suicide,1.0
10551,0.000099,0.016614,0.861426,0.121834,6.258979e-11,1.508402e-06,7.227624e-07,3.615605e-08,4.817902e-07,4.557118e-13,0.000014,3.139150e-10,0.000010,questioncan gun head actually commit self-kill...,suicide,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15501,0.002397,0.005756,0.009222,0.030077,8.784789e-02,2.400540e-02,6.497844e-02,2.052644e-01,2.473162e-01,1.651392e-02,0.098314,1.530452e-04,0.208154,good looking bad personality sucks repost foru...,suicide,1.0
15502,0.000002,0.907711,0.013332,0.075088,4.037281e-09,1.798450e-03,9.147206e-05,3.396475e-05,1.731024e-04,1.433607e-10,0.000023,1.370988e-05,0.001733,tiredi successful career friends sex life issu...,suicide,1.0
15503,0.000002,0.907711,0.013332,0.075088,4.037281e-09,1.798450e-03,9.147206e-05,3.396475e-05,1.731024e-04,1.433607e-10,0.000023,1.370988e-05,0.001733,tiredi successful career friends sex life issu...,suicide,1.0
15504,0.000865,0.086933,0.006027,0.906138,3.021887e-07,2.688924e-07,6.524493e-07,9.087304e-09,6.513253e-07,2.146679e-08,0.000004,8.534478e-07,0.000030,tell parents im strugglingi want get help im ...,suicide,1.0


In [55]:
df_suicide_ag.to_csv('result/df_suicide_train_ag.csv',index=False)

In [220]:
df_suicide_ag['class'].value_counts()

class
suicide        10238
non-suicide     5268
Name: count, dtype: int64

In [221]:
LR_with_emo_feature(df_suicide_ag,y_test_sui,X_test_sui,if_numerical = False)

Accuracy 0.9264315510049298
Precision 0.9264499814776039
Recall 0.9265839610101905
F1-score： 0.9264268850829415


('              precision    recall  f1-score   support\n\n     suicide       0.94      0.92      0.93      1342\n non-suicide       0.92      0.94      0.93      1295\n\n    accuracy                           0.93      2637\n   macro avg       0.93      0.93      0.93      2637\nweighted avg       0.93      0.93      0.93      2637\n',
 array(['non-suicide', 'suicide', 'suicide', ..., 'suicide', 'suicide',
        'suicide'], dtype=object),
 array([[0.97521168, 0.02478832],
        [0.16679508, 0.83320492],
        [0.02124096, 0.97875904],
        ...,
        [0.19313282, 0.80686718],
        [0.02363518, 0.97636482],
        [0.26203083, 0.73796917]]))

In [56]:
pd.concat([X_test_sui,y_test_sui],axis = 1).to_csv('result/df_suicide_test.csv',index = False)